<a href="https://colab.research.google.com/github/EvgenyEsin/BigData/blob/main/DZ_big_data_sem_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашняя работа по Урок 2. SQL & Big Data

In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od
import pandas as pd
import time
from pathlib import Path

## Скачиваем датасет

In [3]:
dataset_path = Path('new-york-city-airbnb-open-data')
if not dataset_path.is_file():
    od.download('https://www.kaggle.com/datasets/dgomonov/new-york-city-airbnb-open-data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: zigzagmed
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/dgomonov/new-york-city-airbnb-open-data


100%|██████████| 2.44M/2.44M [00:00<00:00, 161MB/s]

In [9]:
df = pd.read_csv('new-york-city-airbnb-open-data/AB_NYC_2019.csv')
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [11]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,4.889500e+04,4.889500e+04,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,38843.000000,48895.000000,48895.000000
mean,1.901714e+07,6.762001e+07,40.728949,-73.952170,152.720687,7.029962,23.274466,1.373221,7.143982,112.781327
std,1.098311e+07,7.861097e+07,0.054530,0.046157,240.154170,20.510550,44.550582,1.680442,32.952519,131.622289
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.471945e+06,7.822033e+06,40.690100,-73.983070,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,1.967728e+07,3.079382e+07,40.723070,-73.955680,106.000000,3.000000,5.000000,0.720000,1.000000,45.000000
75%,2.915218e+07,1.074344e+08,40.763115,-73.936275,175.000000,5.000000,24.000000,2.020000,2.000000,227.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


In [5]:
# найдем реднее значение по признаку ”price”
df['price'].mean()

152.7206871868289

In [6]:
# найдем дисперсию
df['price'].var()

57674.02524696099

Используя Python, реализуйте скрипт mapper.py и reducer.py для расчета

In [7]:
def mapper(path):

    if path.is_file():

        result = []

        with open(path, 'r', encoding='utf-8') as f:
            next(f)
            for line in f:
                data = line.strip().split(',')

                if len(data) > 9:
                    try:
                        price = int(data[9])
                        result.append(f"{price}\t1")
                    except ValueError:
                        continue
    else:
        print('Not found file')
    return result


def reducer(input_data):

    sum_price = 0
    count = 0
    prices = []

    for line in input_data:
        price, cnt = line.strip().split('\t')

        try:
            price = int(price)
            cnt = int(cnt)
        except ValueError:
            continue

        sum_price += price * cnt
        count += cnt
        prices.append(price)


    if count == 0:
        print("Нет данных для расчета.")
        return None
    else:
        mean_price = sum_price / count
        variance = sum((x - mean_price) ** 2 for x in prices) / count

        return mean_price, variance

In [8]:
mapped_data = mapper(Path('new-york-city-airbnb-open-data/AB_NYC_2019.csv'))
mean_price, variance = reducer(mapped_data)
print(mean_price, variance)

153.23297422544968 59124.1250938161
